In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('./data/play_tennis.csv')
data.head()
data.drop(columns =  ['day'],inplace = True)

In [4]:
data

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [5]:
data['play'].value_counts()

play
Yes    9
No     5
Name: count, dtype: int64

In [6]:
py = 9/14
pn = 5/14

In [7]:
print(py,pn,sep='\n')

0.6428571428571429
0.35714285714285715


In [8]:
data

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [9]:
results = {}
for col_index in range(data.shape[1]-1):
  a = pd.crosstab(data[data.columns[col_index]],data['play'])
  total_no = a['No'].sum()
  total_yes = a['Yes'].sum()

  for i in range(a.shape[0]):
      row_label = a.index[i]

      results[f'P{row_label}Yes'] = a.loc[row_label, 'Yes'] / total_yes
      results[f'P{row_label}No'] = a.loc[row_label, 'No'] / total_no

print(results)

{'POvercastYes': 0.4444444444444444, 'POvercastNo': 0.0, 'PRainYes': 0.3333333333333333, 'PRainNo': 0.4, 'PSunnyYes': 0.2222222222222222, 'PSunnyNo': 0.6, 'PCoolYes': 0.3333333333333333, 'PCoolNo': 0.2, 'PHotYes': 0.2222222222222222, 'PHotNo': 0.4, 'PMildYes': 0.4444444444444444, 'PMildNo': 0.4, 'PHighYes': 0.3333333333333333, 'PHighNo': 0.8, 'PNormalYes': 0.6666666666666666, 'PNormalNo': 0.2, 'PStrongYes': 0.3333333333333333, 'PStrongNo': 0.6, 'PWeakYes': 0.6666666666666666, 'PWeakNo': 0.4}


In [10]:
input_data = ['Sunny','Hot','High','Weak']

In [ ]:
p_yes = py
p_no = pn

for i in range(len(input_data)):
  value = input_data[i]
  y_str = f'P{value}Yes'
  n_str = f'P{value}No'
  p_yes *= results[y_str]
  p_no *= results[n_str]

print('no' if p_no > p_yes else 'yes')

In [ ]:
import pandas as pd

def fit(data, target_col='play'):
  results = {}
  feature_cols = [col for col in data.columns if col != target_col]

  for col in feature_cols:
    crosstab = pd.crosstab(data[col], data[target_col])
    total_no = crosstab['No'].sum()
    total_yes = crosstab['Yes'].sum()

    for val in crosstab.index:
      results[f'P{val}Yes'] = crosstab.loc[val, 'Yes'] / total_yes
      results[f'P{val}No'] = crosstab.loc[val, 'No'] / total_no

  return results

def predict(input_data):
  py,pn = data['play'].value_counts()/data.shape[0]
  for i in range(len(input_data)):
    y_str = f'P{input_data[i]}Yes'
    n_str = f'P{input_data[i]}No'
    py *= results[y_str]
    pn *= results[n_str]

  print('no' if py > pn else 'yes')


In [23]:
import pandas as pd

class SimpleNaiveBayes:
  def __init__(self):
    self.results = {}
    self.prior_yes = None
    self.prior_no = None

  def fit(self, data, target_col='play'):
    self.results = {}
    feature_cols = [col for col in data.columns if col != target_col]

    counts = data[target_col].value_counts(normalize=True)
    self.prior_yes = counts.get('Yes', 0)
    self.prior_no = counts.get('No', 0)

    for col in feature_cols:
      crosstab = pd.crosstab(data[col], data[target_col])
      total_no = crosstab['No'].sum()
      total_yes = crosstab['Yes'].sum()

      for val in crosstab.index:
        self.results[f'P{val}Yes'] = crosstab.loc[val, 'Yes'] / total_yes
        self.results[f'P{val}No'] = crosstab.loc[val, 'No'] / total_no

  def predict(self, input_data):
    p_yes = self.prior_yes
    p_no = self.prior_no

    for val in input_data:
      y_key = f'P{val}Yes'
      n_key = f'P{val}No'
      p_yes *= self.results.get(y_key, 1e-6)
      p_no *= self.results.get(n_key, 1e-6)

    return 'no' if p_no > p_yes else 'yes'


In [28]:
bayes = SimpleNaiveBayes()
bayes.fit(data)
bayes.predict(['Rain','Cool','Low','Strong'])

'yes'